In [10]:
!pip install konlpy
!pip install bs4

import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import re

################    크롤링  #################

twitter = Okt()
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" \
      "code=181414&type=after&isActualPointWriteExecute=false&" \
      "isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}"


# 클리닝 함수
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    return cleaned_text

def get_reple(page=1):
    response = requests.get(url.format(page))
    soup = BeautifulSoup(response.text, 'html.parser')
    s, t = [], []

    for li in soup.find('div', {'class': 'score_result'}).find_all('li'):
        # text를 가져올때 공백을 제거한다
        str = li.p.get_text(" ", strip=True)
        str = clean_text(str)

        # 8점이상 좋은영화
        if int(li.em.text) >= 8:
            s.append(1)
            t.append(str)
        # 5점 이하 좋지 않은영화
        elif int(li.em.text) <= 5:
            s.append(0)
            t.append(str)
    return s, t


score, text = [], []

# 1~150page까지 불러온 후 배열에 넣어준다.
for i in range(1, 150):
    print('요청 횟수:', i)
    s, t = get_reple(i)
    # 배열을 합하여준다.
    score += s
    text += t

df = pd.DataFrame([score, text]).T
df.columns = ['score', 'text']
# csv파일로 저장한다.
df.to_csv('test.csv', encoding='utf-8-sig')

################    여기까지 크롤링  #################

요청 횟수: 1
요청 횟수: 2
요청 횟수: 3
요청 횟수: 4
요청 횟수: 5
요청 횟수: 6
요청 횟수: 7
요청 횟수: 8
요청 횟수: 9
요청 횟수: 10
요청 횟수: 11
요청 횟수: 12
요청 횟수: 13
요청 횟수: 14
요청 횟수: 15
요청 횟수: 16
요청 횟수: 17
요청 횟수: 18
요청 횟수: 19
요청 횟수: 20
요청 횟수: 21
요청 횟수: 22
요청 횟수: 23
요청 횟수: 24
요청 횟수: 25
요청 횟수: 26
요청 횟수: 27
요청 횟수: 28
요청 횟수: 29
요청 횟수: 30
요청 횟수: 31
요청 횟수: 32
요청 횟수: 33
요청 횟수: 34
요청 횟수: 35
요청 횟수: 36
요청 횟수: 37
요청 횟수: 38
요청 횟수: 39
요청 횟수: 40
요청 횟수: 41
요청 횟수: 42
요청 횟수: 43
요청 횟수: 44
요청 횟수: 45
요청 횟수: 46
요청 횟수: 47
요청 횟수: 48
요청 횟수: 49
요청 횟수: 50
요청 횟수: 51
요청 횟수: 52
요청 횟수: 53
요청 횟수: 54
요청 횟수: 55
요청 횟수: 56
요청 횟수: 57
요청 횟수: 58
요청 횟수: 59
요청 횟수: 60
요청 횟수: 61
요청 횟수: 62
요청 횟수: 63
요청 횟수: 64
요청 횟수: 65
요청 횟수: 66
요청 횟수: 67
요청 횟수: 68
요청 횟수: 69
요청 횟수: 70
요청 횟수: 71
요청 횟수: 72
요청 횟수: 73
요청 횟수: 74
요청 횟수: 75
요청 횟수: 76
요청 횟수: 77
요청 횟수: 78
요청 횟수: 79
요청 횟수: 80
요청 횟수: 81
요청 횟수: 82
요청 횟수: 83
요청 횟수: 84
요청 횟수: 85
요청 횟수: 86
요청 횟수: 87
요청 횟수: 88
요청 횟수: 89
요청 횟수: 90
요청 횟수: 91
요청 횟수: 92
요청 횟수: 93
요청 횟수: 94
요청 횟수: 95
요청 횟수: 96
요청 횟수: 97
요청 횟수: 98
요청 횟수: 99
요청 횟수: 100
요청 횟수: 1

In [11]:
###############     LogisticRegression을 이용한 감정분석 with TF-IDF,BOW(TfidfVectorizer사용)    #################
# TF-IDF란? https://nesoy.github.io/articles/2017-11/tf-idf
# BOW란? url1 :: https://medium.com/@omicro03/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-7%EC%9D%BC%EC%B0%A8-bag-of-words-20b2af01d56f
#        url2 :: https://ldabook.com/word-representations.html
# TfidfVectorizer란? https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/
# df = pd.read_csv('test.csv', encoding='utf-8')
# x값: bow로 벡터화한값, y: 영화 평점
text = df.get('text')
score = df.get('score')

# 트레이닝/테스트셋을 분리한다.
# random_state: random으로 나누어준다, 0은 항상 같은결과가 나오도록 seed고정
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(text, score, test_size=0.2, random_state=0)
train_y = train_y.astype('float')
test_y = test_y.astype('float')
type(train_x)
print(train_x)
print(train_y)

758     개연성이 떨어지고 스토리 완성도도 떨어진다  너무 스토리가 엉망이라 판타지인줄 알았...
938                                  관람객 긴장감 쩔어서 재밌게 봤습니당
10      기대 진짜 많이하고갔는데 큰 의미를 부여하는건 좋았지만 그걸 지루하지않고 쉽게 풀어...
991     ㅈ평점왜케낮아졌지 ㅋㅋㅋ 마지막 1분을 위한 영화 또 중간중간에 장치들이 많아서 해...
1112    평론가들 너도나도 빨아재끼는 문화적 허영심 역겹다 그래 미국의 모순적인 부분과 상징...
                              ...                        
763                             다들 연기도 잘하고 재밌었다ㅋㅋ 역시 기대이상
835                           재밌었음 스토리도 연출도 배우들 연기가 장난 아님
1216                                                어스겟아웃
559            믿을수없는 평점임 2점 주는것도 아까움 도플갱어가 입여는순간부터 망한영화느낌
684                                    넘나 지루하고 내용을 잘모르겠네요
Name: text, Length: 1016, dtype: object
758     0.0
938     1.0
10      0.0
991     1.0
1112    0.0
       ... 
763     1.0
835     1.0
1216    1.0
559     0.0
684     0.0
Name: score, Length: 1016, dtype: float64


In [12]:
# 단어들을 모아 빈도수 체크 (TF-IDF)
# feature(문장의 특징) 노출수를 가중치로 설정한 Bag Of Word 벡터를 만든다.
from sklearn.feature_extraction.text import TfidfVectorizer

# ngram으로 단어를 n개로 잘라서 보겠다는 의미 (1, 2그램 사용했다)
# ex) 재밌다. 너무 재밌다. 노잼. 완전 노잼.
# tokenizer는 twitter.morphs사용
# max_df: 너무 많은단어들(the, a)제외
# min_df: 너무적은단어 제외 몇만개에서 3개만 나오는 단어의 경우 의미가 없을 가능성이 높다
# https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/
tfv = TfidfVectorizer(tokenizer=twitter.morphs, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x

<1016x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 14536 stored elements in Compressed Sparse Row format>

In [13]:
## 참고(없어도 학습가능) tf-idf 형태 ##
#  tfidf 값을 가중치로 설정한 행렬을 볼 수 있다.
# 부정예시 : 돈 아깝다
features = tfv.get_feature_names()
stm = np.asarray(tfv_train_x.toarray())
df = pd.DataFrame(stm)
df.columns = features
print(df)

        0  0 점    1  1 도  1 위  1 점  1 점도   10  10 점   15  ...  후반 부  후반 부로  \
0     0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
...   ...  ...  ...  ...  ...  ...   ...  ...   ...  ...  ...   ...    ...   
1011  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
1012  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
1013  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
1014  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   
1015  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0   

       후회    훨   훨씬    휴   흑인    흠  흥미진진    힘  
0     0.0  0.0 

In [14]:
# 이진분류 알고리즘(로지스틱 회귀)
from sklearn.linear_model import LogisticRegression
# 하이퍼파라미터 최적화
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C': [0.1, 1, 0.01, 3, 5, 10]}

# params중 정확도가 가장 높은 param을 선택
# cv=4 : 4번의 모의를 통해 가장 좋은 params를 선택
# https://datascienceschool.net/view-notebook/ff4b5d491cc34f94aea04baca86fbef8/ 참조
# verbose: 상세정보(숫자가 클수록 상세정보가 자세히 보여진다)
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring='accuracy', verbose=1)

# 최적의 하이퍼파라미터를 사용하여 학습
grid_cv.fit(tfv_train_x, train_y)
# grid_cv: 학습한 모델에 접근할 수 있도록 PREDICT와 SCORE 메서드 제공

Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Anaconda3\envs\mytest\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.0s finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 0.01, 3, 5, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [15]:
# 가장 좋은 parameter
print(grid_cv.best_params_)
# 가장 좋은 parameter
print(grid_cv.best_score_)

{'C': 10}
0.8208661417322834


In [16]:
# test data 학습

# 벡터변환기
tfv_test_x = tfv.transform(test_x)
# 정확도 예측 (train data와 test data의 정확도를 비교하며 over fitting에 빠졌는지 확인해준다)
grid_cv.best_estimator_.score(tfv_test_x, test_y)

0.7795275590551181

In [26]:
# 만들어진 모델로 영화 긍부정 구분

# 긍정 테스트
#ex = ['진짜 재밌어요! 강추!!']

# 부정 테스트
ex = ['좀 그랬어요']
# 감정분석사전과는 달리 비속어도 구분이 가능하다
# 하지만 라벨링이 되어있지않은 소셜데이터의경우 감정분석사전을 사용하는게 더 좋을 수 있다.
my_review = tfv.transform(ex)

# 0: 부정, 1: 긍정
if grid_cv.best_estimator_.predict(my_review) == 0:
    print("부정")
else:
    print("긍정")
print(grid_cv.best_estimator_.predict(my_review))

부정
[0.]


In [18]:
## 참고 (없어도 학습가능) 입력 tf-idf 형태 ##
features = tfv.get_feature_names()
stm = np.asarray(my_review.toarray())
df = pd.DataFrame(stm)
df.columns = features
print(df)

     0  0 점    1  1 도  1 위  1 점  1 점도   10  10 점   15  ...  후반 부  후반 부로   후회  \
0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0    0.0  0.0   

     훨   훨씬    휴   흑인    흠  흥미진진    힘  
0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  

[1 rows x 1472 columns]
